In [ ]:
from sympy import symbols, cos, sin, diff, Function, simplify

# Define symbols
theta = symbols('theta')
v0, b, a, k, e, h, GM, sigma = symbols('v0 b a k e h GM sigma')

# Define r(theta)
r = a*(e**2-1) / (1 + e* cos(theta/sigma))

# Calculate dr/dtheta
dr_dtheta = diff(r, theta)

# Define omega(theta) = dtheta/dt
omega = (v0*b) / r**2

# Calculate d^2r/dtheta^2
d2r_dtheta2 = diff(dr_dtheta, theta)

# Calculate domega/dtheta
domega_dtheta = diff(omega, theta)

dr_dt = diff(r,theta)*omega
# Calculate d^2r/dt^2 using chain rule
d2r_dt2 = simplify(d2r_dtheta2 * omega**2 + dr_dtheta * domega_dtheta * omega)

# Calculate d^2theta/dt^2
# Differentiate omega(theta)
d2theta_dt2 = simplify(domega_dtheta * omega)

# Display the results
print(f"omega= {omega}")
print(f"r = {r}")
print(f"dr_dt= {dr_dt}")
# print(f"dr_dtheta = {dr_dtheta}")
print(f"d2r_dtheta2 = {d2r_dtheta2}")
print(f"domega_dtheta = {domega_dtheta}")
print(f"d2r_dt2 = {d2r_dt2}")
print(f"d2theta_dt2 = {d2theta_dt2}")


In [ ]:
import numpy as np
from scipy.optimize import minimize
from scipy.integrate import quad
import matplotlib.pyplot as plt
from astropy import constants as cc, units as uu



def calc_ab(v0,GM, e):
    a = GM/(v0**2)
    b= a*np.sqrt(e**2-1)
    return a, b

def calc_ae(v0,GM, b):
    a = GM/(v0**2)
    e = np.sqrt(1+(b/a)**2)
    return a, e

# Constants
G = cc.G  # gravitational constant in m^3 kg^-1 s^-2
M = cc.M_sun  # mass of the Sun in kg
c = cc.c.si.value  # speed of light in m/s
GM = (G*M).si.value  # gravitational parameter for the Sun in m^3/s^2

# Initial conditions
m_sun = cc.M_sun.si.value
r_sun = cc.R_sun.si.value
# Constants for the hyperbola
v0 = c
b= 20*r_sun
a, e = calc_ae(v0, GM,b)
print(a/r_sun, e)




def r_derivatives(theta, sigma):
    # Expression for r, r_dot, and r_double_dot
        omega= b*v0*(e*np.cos(theta/sigma) + 1)**2/(a**2*(e**2 - 1)**2)
        r = a*(e**2 - 1)/(e*np.cos(theta/sigma) + 1)
        dr_dt= b*e*v0*np.sin(theta/sigma)/(a*sigma*(e**2 - 1))
        d2r_dt2 = b**2*e*v0**2*(e*np.cos(theta/sigma) + 1)**2*np.cos(theta/sigma)/(a**3*sigma**2*(e**2 - 1)**3)
        return r, dr_dt, d2r_dt2, omega

def calc_r(theta, a, e):
    r = a*(e**2 - 1)/(e*np.cos(theta) + 1)
    return r


def newtonian_grav_accel(GM, r):
    a_theoretical = -GM / r**2
    return a_theoretical

def new_grav_accel(GM, r, r_dot):
    alpha = np.sqrt(2*GM/(np.abs(r)*c**2)) # alpha is alreayd v/c
    v = alpha*c
    gamma_v = 1 / np.sqrt(1 -alpha**2)
    B = (1 + (gamma_v - 1) * (r_dot / v)**2)
    a_theoretical = -GM / gamma_v / r**2 / B
    return a_theoretical

def error_functionHU(x):
    sigma = x
    theta_inf = np.arccos(-1 / e)
    def integrand(theta):
        r, r_dot, r_double_dot, omega = r_derivatives(theta,sigma)
        a_theoretical = new_grav_accel(GM, r, r_dot)
        a_numerical = r_double_dot - r**2*omega # Assuming r_double_dot is defined to calculate \ddot{r}
        return (a_theoretical - a_numerical)**2

    integral_error, _ = quad(integrand, -theta_inf, theta_inf, epsabs=1e-5, epsrel=1e-5)  # Use the correct limits
    return 1E6*integral_error

def error_functionNewton(x):
    sigma = x
    theta_inf = np.arccos(-1 / e)
    def integrand(theta):
        r, r_dot, r_double_dot, omega =  r_derivatives(theta,sigma)
        a_theoretical = newtonian_grav_accel(GM, r)
        a_numerical = r_double_dot - r**2*omega  # Assuming r_double_dot is defined to calculate \ddot{r}
        return (a_theoretical - a_numerical)**2

    integral_error, _ = quad(integrand, -theta_inf, theta_inf, epsabs=1e-5, epsrel=1e-5)  # Use the correct limits
    return 1E6*integral_error

def optimize_trajectory(errf):
    initial_guess = [1]  # Initial guesses for sigma and e
    result = minimize(errf, initial_guess, method='Nelder-Mead', tol=1e-6)
    delta = result.x
    optimized_e = e
 
    print("delta =", delta)

    # Calculate the deflection angle Δθ in radians
    theta_inf = np.arccos(-1 / e)
    delta_theta = - np.pi - 2*theta_inf 
    delta_theta_degrees = np.degrees(delta_theta)
    print(f"The deflection angle Δθ is {delta_theta:.6}")
    return delta_theta, delta


delta_theta, delta = optimize_trajectory(error_functionHU)
# optimize_trajectory(error_functionHU)
print("delta_theta, delta", delta_theta, delta)
optimized_e = e
optimized_v0 = v0
optimized_a, optimized_b = calc_ab(optimized_v0,GM,optimized_e)

# Function to calculate r based on theta

# Generate theta values from -theta_inf to theta_inf
theta_inf = np.arccos(-1 / e)
# p = 20*a
# theta_inf = np.arccos(1-a/p/(e**2-1))
theta_values = np.linspace(-theta_inf, theta_inf, 1000)

# Plotting the hyperbola
plt.figure(figsize=(8, 6))



r_values = calc_r(theta_values, a, e)
x_values = r_values * np.cos(theta_values)
y_values = r_values * np.sin(theta_values)

plt.plot(x_values, y_values, label=f'CalcHyperbola: a={a}, e={e}')
# Calculate r for each theta
# Convert polar coordinates to Cartesian for plotting
x_values = r_values * np.cos(theta_values)
y_values = r_values * np.sin(theta_values)

plt.plot(x_values, y_values, label=f'Hyperbola: a={a}, e={e}')
plt.scatter([0], [0], color='yellow', label='Central Body')  # Central body (e.g., the Sun)
plt.title('Hyperbolic Trajectory around a Central Body')
plt.xlabel('x-coordinate')
plt.ylabel('y-coordinate')
plt.axhline(0, color='grey', linestyle='--')  # Asymptote
plt.axvline(0, color='grey', linestyle='--')  # Asymptote
plt.grid(True)
plt.legend()
plt.axis('equal')  # Ensure equal scaling for x and y axes
plt.show()



In [ ]:
import matplotlib.pyplot as plt
import numpy as np


# Function to calculate r based on theta
def calc_r(theta, a, e):
    return  a * (e**2 - 1) / (1 + e * np.cos(theta))


# Generate theta values from -theta_inf to theta_inf
theta_inf = np.arccos(-1 / e)
# p = 20*a
# theta_inf = np.arccos(1-a/p/(e**2-1))
theta_values = np.linspace(-theta_inf, theta_inf, 1000)

# Plotting the hyperbola
plt.figure(figsize=(8, 6))


r_calc = calc_r(theta_values, a, e)
x_values = r_calc * np.cos(theta_values)
y_values = r_calc * np.sin(theta_values)

plt.plot(x_values, y_values, label=f'CalcHyperbola: a={optimized_a}, e={optimized_e}')
# Calculate r for each theta
r_values = calc_r(theta_values, optimized_a, optimized_e)
# Convert polar coordinates to Cartesian for plotting
x_values = r_values * np.cos(theta_values)
y_values = r_values * np.sin(theta_values)

plt.plot(x_values, y_values, label=f'Hyperbola: a={a}, e={e}')
plt.scatter([0], [0], color='yellow', label='Central Body')  # Central body (e.g., the Sun)
plt.title('Hyperbolic Trajectory around a Central Body')
plt.xlabel('x-coordinate')
plt.ylabel('y-coordinate')
plt.axhline(0, color='grey', linestyle='--')  # Asymptote
plt.axvline(0, color='grey', linestyle='--')  # Asymptote
plt.grid(True)
plt.legend()
plt.axis('equal')  # Ensure equal scaling for x and y axes
plt.show()


In [ ]:




# r1, dr_dt1, d2r_dt21, omega =r_derivatives(theta_values, sigma)
r1, dr_dt1, d2r_dt21, omega =r_derivatives(theta_values, 1)
newton_accel = newtonian_grav_accel(GM, r1)
a_numerical = d2r_dt21 - r1*omega**2

# bb = a_numerical[100:120]/newton_accel[100:120]


# plt.plot(theta_values[100:120], bb, color="black")
plt.plot(theta_values, newton_accel-200, color="black")
# plt.plot(theta_values, HU_acell+0.00001, color="red")
plt.plot(theta_values, a_numerical, color="green")
plt.show()

# Summing up waves

In [ ]:
from sympy import symbols, cos, sin, pi, atan2, simplify, expand
from sympy.vector import CoordSys3D

# Define symbols
k1_r, k1_theta, lambda_, a1, b1, alpha, N, R, omega, t, K, beta= symbols('k1_r k1_theta lambda_ a1 b1 alpha N R omega t K beta')

# Define the coordinate system
N2 = CoordSys3D('N2')

# Vectors
r_vector = a1 * N2.i + b1 * N2.j  # Position vector
k2_r_vector = N2.i  # Radial unit vector in the direction of r
k1_r_vector = k1_r*N2.i  # Radial unit vector in the direction of r
k1_theta_vector = k1_theta*N2.j  # Angular unit vector perpendicular to r

# Wave functions without the time term
psi_1 = cos((2 * pi / lambda_) * r_vector.dot(k1_r_vector + k1_theta_vector))
psi_2 = cos((2 * pi / lambda_) * r_vector.dot(k2_r_vector))

# Since a2 has been dropped and b2 absorbed into the constants, we have only one direction which is k_r
# We will express 1/K as the cosine of an angle beta, where beta is defined through the expression for K


# Expression for Psi, combining the wave functions and accounting for the normalization factor K
Psi = K * (cos(beta) * psi_1 + sin(beta) * psi_2)

# Simplify Psi
simplified_Psi = expand(Psi)

simplified_Psi


In [ ]:
beta = atan2(2 * pi * alpha * N / R, K)

In [ ]:
from sympy import symbols, cos, sin, pi, atan2, simplify, expand
from sympy.vector import CoordSys3D

# Define symbols
k1_r, k1_theta, lambda_, a1, b1, alpha, N, R, K, beta = symbols('k1_r k1_theta lambda_ a1 b1 alpha N R K beta')

# Define the coordinate system
N2 = CoordSys3D('N2')

# Vectors
r_vector = a1 * N2.i + b1 * N2.j  # Position vector in terms of its components
k1_r_vector = k1_r * N2.i  # k_r vector component
k1_theta_vector = k1_theta * N2.j  # k_theta vector component

# Wave functions without the time term
psi_1 = cos((2 * pi / lambda_) * (k1_r_vector + k1_theta_vector).dot(r_vector))
psi_2 = sin(beta) * cos((2 * pi / lambda_) * k1_r_vector.dot(r_vector))

# Combine the wave functions with the normalization factor K
Psi = K * (cos(beta) * psi_1 + psi_2)

# Expand and simplify the expression for Psi
simplified_Psi = expand(Psi)
simplified_Psi

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# Constants
G = 6.67430e-11  # Gravitational constant, m^3 kg^-1 s^-2
M = 1.98847e30   # Mass of the Sun, kg
c = 299792458    # Speed of light, m/s

# Define the system of ODEs
def photon_motion(t, y):
    # y = [r, theta, dr/dt, dtheta/dt]
    r = y[0]
    theta = y[1]
    drdt = y[2]
    dthetadt = y[3]  # This will be defined by the angular momentum conservation

    # Calculate d^2r/dt^2 and d^2theta/dt^2 (you'll provide these expressions)
    d2rdt2 = -G * M / r**2  # This is a placeholder, replace it with your expression
    # For d2thetadt2, we'll assume angular momentum conservation, so it's derived from that
    d2thetadt2 = ... # Your expression here

    # Return the derivatives as a list
    return [drdt, dthetadt, d2rdt2, d2thetadt2]

# Initial conditions: [r0, theta0, dr/dt0, dtheta/dt0]
initial_conditions = [ ... ]  # Fill in with the initial conditions for the problem

# Time span for the integration (t0, tf)
t_span = (0, ... )  # Fill in with the start and end times for the integration

# Call the integrator
sol = solve_ivp(photon_motion, t_span, initial_conditions, method='RK45')

# Extract the results
r = sol.y[0]
theta = sol.y[1]

# Plot the trajectory (r vs theta)
plt.polar(theta, r)
plt.show()
